In [ ]:
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Input
from tensorflow.keras.models import Model

# Define the BiLSTM model
input_seq = Input(shape=(max_len, len(amino_acid_mapping)))

# Define Bidirectional LSTM layers
x = Bidirectional(LSTM(256, return_sequences=True))(input_seq)
x = Bidirectional(LSTM(256))(x)

# Broad output
broad_output = Dense(len(broad_label_encoder.classes_), activation='softmax', name='broad_output')(x)

# Detailed output
detailed_output = Dense(len(protein_label_encoder.classes_), activation='softmax', name='detailed_output')(x)

# Create and compile the model
model_bilstm = Model(inputs=input_seq, outputs=[broad_output, detailed_output])
model_bilstm.compile(optimizer='adam',
                     loss={'broad_output': 'categorical_crossentropy', 'detailed_output': 'categorical_crossentropy'},
                     metrics={'broad_output': 'accuracy', 'detailed_output': 'accuracy'})

# Model summary
model_bilstm.summary()

# Train the BiLSTM model
history_bilstm = model_bilstm.fit(
    X_train,
    {'broad_output': y_broad_train, 'detailed_output': y_train},
    epochs=10,  # Adjust epochs
    batch_size=32,  # Adjust batch size
    validation_split=0.2,
    callbacks=[early_stopping]  # Using EarlyStopping as before
)

# Evaluate the model
model_bilstm.evaluate(X_test, [y_broad_test, y_test])
